## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-01-31-32 +0000,nyt,Israel Stages Rare Attack on Syrian Government...,https://www.nytimes.com/2025/07/15/world/middl...
1,2025-07-16-01-06-47 +0000,bbc,Trump launches probe into Brazil's 'unfair' tr...,https://www.bbc.com/news/articles/cj0mg2j7z04o
2,2025-07-15-23-48-34 +0000,nyt,Huckabee Calls Death of Palestinian-American i...,https://www.nytimes.com/2025/07/15/world/middl...
3,2025-07-15-21-33-38 +0000,nyt,What to Know About the Fighting in Southern Syria,https://www.nytimes.com/2025/07/15/world/asia/...
4,2025-07-15-21-10-32 +0000,nyt,Wednesday Briefing: China’s Economy Grows Desp...,https://www.nytimes.com/2025/07/15/briefing/ch...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


,word,score
7,trump,19
30,tariffs,8
73,ukraine,7
27,china,7
91,new,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
73,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...,57
39,2025-07-15-13-32-31 +0000,nyt,China’s Economy Grows Steadily Despite Trump’s...,https://www.nytimes.com/2025/07/14/business/ch...,40
54,2025-07-15-11-15-55 +0000,nyt,E.U. Nations Plan Retaliatory Tariffs Against ...,https://www.nytimes.com/2025/07/15/world/europ...,39
59,2025-07-15-09-38-38 +0000,bbc,'Not our war' - Trump's Nato weapons deal for ...,https://www.bbc.com/news/articles/c14e2ydv4d6o,39
70,2025-07-15-04-30-50 +0000,bbc,China growth beats expectations as Trump tarif...,https://www.bbc.com/news/articles/c20r461g61xo,39


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
73,57,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...
34,26,2025-07-15-14-57-21 +0000,bbc,Two dead after flash flooding in New Jersey an...,https://www.bbc.com/news/articles/c0j42xy47q7o
22,24,2025-07-15-17-49-34 +0000,bbc,Carney says US-Canada trade deal likely to inc...,https://www.bbc.com/news/articles/c0q82qqxegno
40,22,2025-07-15-13-09-23 +0000,nyt,Zelensky Announces Plan for Major Government S...,https://www.nytimes.com/2025/07/14/world/europ...
17,18,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
67,16,2025-07-15-06-42-06 +0000,bbc,"As theories swirl about Air India crash, key d...",https://www.bbc.com/news/articles/cp3lpdqp7x3o
6,15,2025-07-15-20-41-00 +0000,wsj,"Inflation picked up in June, a potential sign ...",https://www.wsj.com/economy/inflation-hit-2-7-...
23,15,2025-07-15-17-14-09 +0000,bbc,Thousands of Afghans were moved to UK in secre...,https://www.bbc.com/news/articles/cvg8zy78787o
21,12,2025-07-15-18-11-29 +0000,bbc,French PM looks to scrap two public holidays i...,https://www.bbc.com/news/articles/c2d02dz6gedo
13,12,2025-07-15-20-07-27 +0000,nyt,Police Arrest Far-Right Leader After Anti-Immi...,https://www.nytimes.com/2025/07/15/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
